In [1]:
import pandas as pd
import seaborn as sns 
import scipy as sp 
from  matplotlib import pyplot as plt
sns.set(font_scale=1.1, style="white",  #palette="viridis", 
        rc={'font.size': 16, 'figure.figsize': (7,4), 'axes.grid': True, 'lines.linewidth':2.0, 
            'grid.color': '.8', 'grid.linewidth': 0.5,})


In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

In [17]:
import GPy as gpy

In [3]:
import sys
sys.path.append("/Users/ipaulo/yandexDisk/DIPLOMA/CODE/py-src/")
from itr import * 
import rpy2.robjects as ro
import rpy2
# ro.r.source("/Users/ipaulo/yandexDisk/DIPLOMA/CODE/src/clean_sources.R");

itr.py is imported!


In [11]:
from numpy.random import RandomState
prng = RandomState(1234567890)
prng.randint(-1, 2, size=10)

array([ 1,  1, -1,  0,  0, -1,  1,  0, -1, -1])

In [201]:
import go_amp

itr.py is imported!


In [205]:
f = lambda x: (x**2).sum()

itr.py is imported!


In [235]:
go_amp.AMPGO(f, [-1000, 100], maxiter=10 )

(array([-0.00000001, -0.00000001]),
 5.0000008931800974e-17,
 123,
 'Maximum number of function evaluations exceeded',
 (2, 0))

## Read data

In [162]:
X, Y = np.load("../data/sarcos/Xtr.npy"),  np.load("../data/sarcos/Ytr.npy")
rng = np.random.RandomState(777)
n_train, n_test = 1000, 1000
rnd_idxs = rng.randint(0, X.shape[0], n_train + n_test)

Xtr, Ytr = [i[rnd_idxs[:n_train], :] for i in [X, Y]]
Xts, Yts = [i[rnd_idxs[n_train:], :] for i in [X, Y]]
# assert not np.in1d(tr_idxs, ts_idxs).max()

In [ ]:
def plot_multiple_outputs(m,xlim,ylim):
    fig = plt.figure(figsize=(12,8))
    #Output 1
    ax1 = fig.add_subplot(211)
    ax1.set_xlim(xlim)
    ax1.set_title('Output 1')
    m.plot(plot_limits=xlim,fixed_inputs=[(1,0)],which_data_rows=slice(0,100),ax=ax1)
    ax1.plot(Xt1[:,:1],Yt1,'rx',mew=1.5)
    #Output 2
    ax2 = fig.add_subplot(212)
    ax2.set_xlim(xlim)
    ax2.set_title('Output 2')
    m.plot(plot_limits=xlim,fixed_inputs=[(1,1)],which_data_rows=slice(100,200),ax=ax2)
    ax2.plot(Xt2[:,:1],Yt2,'rx',mew=1.5) 

In [ ]:
fig, ax = plt.subplots(5, 3, figsize=(9, 8))
labels = []
i = j = 0
for n_size, grp_size in df.groupby("sample_size"):
    for sc, grp_size_sc in grp_size.groupby("scenario"):
        g = grp_size_sc.groupby("s_factor")
        ms = g.value_f.mean()
        x, m = ms.index, ms.values
        ms.plot(ax=ax[i,j])
        std = g.value_f.std().values
        ax[i,j].fill_between(x, m + std, m - std, color='g', alpha=0.1 )
        if j == 0: ax[i,j].set_ylabel("n = {}".format(n_size))
        if i == 0: ax[i,j].set_title(sc)
        j += 1 
    i += 1
    j = 0 
# plt.tight_layout(pad = 1.01, h_pad=-0.2, w_pad=0.1)


In [107]:
np.set_printoptions(suppress=True)

In [192]:
from itertools import zip_longest

In [198]:
np.hstack([[ "C{}".format(i) for i in range(X.shape[1]) ],
[ "A{}".format(i) for i in range(Y.shape[1]) ],
[ "R" ]])

array(['C0', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10',
       'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19',
       'C20', 'A0', 'A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'R'], 
      dtype='<U3')

### Baseline

In [186]:
from sklearn.linear_model import LinearRegression
from sklearn.base import clone

lin_m = LinearRegression(n_jobs=-1)
models = [clone(lin_m).fit(X, Y[:, i]) for i in range(Y.shape[1])]
np.hstack([m.predict(X).reshape(-1,1) for m in models])

In [ ]:
def get_rewards_for_preds(preds, Y):
    return -np.square(preds - Y).sum(axis=1)

###  Simple miltioutput GP 

In [163]:
k = gpy.kern.RBF(21)
n_restarts = 2 
models = [gpy.models.GPRegression(Xtr, Ytr[:, [i]], kernel=k.copy()) for i in range(Ytr.shape[1])]
[m.optimize_restarts(num_restarts=n_restarts, verbose=False, robust=True) for m in models];

 /Users/ipaulo/anaconda/lib/python3.5/site-packages/GPy/kern/src/stationary.py:160: RuntimeWarning:overflow encountered in true_divide
 /Users/ipaulo/anaconda/lib/python3.5/site-packages/GPy/kern/src/rbf.py:35: RuntimeWarning:overflow encountered in square
 /Users/ipaulo/anaconda/lib/python3.5/site-packages/GPy/kern/src/rbf.py:38: RuntimeWarning:invalid value encountered in multiply
 /Users/ipaulo/anaconda/lib/python3.5/site-packages/paramz/transformations.py:106: RuntimeWarning:invalid value encountered in greater
 /Users/ipaulo/anaconda/lib/python3.5/site-packages/paramz/transformations.py:111: RuntimeWarning:invalid value encountered in greater


In [164]:
def predict_simple_mogp(Xts, models):
    return np.hstack([m.predict(Xts)[0] for m in models ]) 

def get_mogp_score(preds, Yts):
    """return mean reward – aka mean squared euclidian distance"""
    return np.square(Yts - preds).sum(axis=1).mean()

In [165]:
get_mogp_score(predict_simple_mogp(Xts, models), Yts)

42.350970174520036

###  Simple miltioutput GP 

In [166]:
k = gpy.kern.RBF(21)
m = gpy.models.GPRegression(Xtr, Ytr, kernel=k) 
m.optimize_restarts(num_restarts=n_restarts, verbose=False, robust=True);

In [ ]:
get_mogp_score(m.predict(Xts)[0], Yts)

### Coregionalized regression

In [ ]:
kernel = GPy.util.multioutput.ICM(input_dim=1,num_outputs=2,kernel=GPy.kern.Bias(input_dim=1))
m = GPy.models.GPCoregionalizedRegression(X_list=[X1,X2],Y_list=[Y1,Y2],kernel=kernel)
m['.*bias.var'].constrain_fixed(1) #B.kappa now encodes the variance.
m['.*W'].constrain_fixed(0)
m.optimize()

In [ ]:
K = GPy.kern.Matern32(1)
icm = GPy.util.multioutput.ICM(input_dim=1,num_outputs=2,kernel=K)

m = GPy.models.GPCoregionalizedRegression([X1,X2],[Y1,Y2],kernel=icm)
m['.*Mat32.var'].constrain_fixed(1.) #For this kernel, B.kappa encodes the variance now.
m.optimize()

In [111]:
from itertools import  repeat

In [ ]:
n_output = 7
icm = gpy.util.multioutput.ICM(input_dim=21,num_outputs=n_output,kernel=gpy.kern.RBF(21))
m = gpy.models.GPCoregionalizedRegression(list(repeat(Xtr, n_output)), np.split(Ytr, 7, axis=1), kernel=icm)
m.optimize()
# prediction data should include number of component
m.predict(list(repeat(Xtr, n_output))) # this will fail